# Transformación del data set para alimentar modelo YOLO

In [84]:
# !pip install Pillow
# !pip install rawpy
# !pip install imageio
# !pip install yolov5
# !pip install torch torchvision
# !pip install numpy opencv-python
# !pip install roboflow

In [85]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [86]:
import torch
import cv2
import os

Se hace la carga del modelo de YOLOv5 con el uso de la herramiento torch hub para así lograr tener el modelo pre entrenado cargado y listo para ser utilizado.

In [87]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /Users/marcojurado/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-8-22 Python-3.10.5 torch-2.4.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Vamos a definir las categorias que nos van a ser de interes para detección de objetos. Estos son los que para esta investigación y desarrollo vamos a estar trabajando.

In [88]:
classes_of_interest = ['stop', 'traffic light']

Función para detección y recortar los objetos detectados mediante su bounding box para lograr obtener las imagenes resultantes de este proceso.

In [89]:
input_folder = 'datasets/dataset_secundario'

Los folders de output que vamos a estar utilizando

In [90]:
output_folder = 'datasets/cropped_and_labeled'
images_folder = os.path.join(output_folder, 'images')
labels_folder = os.path.join(output_folder, 'labels')

Verificamos si existe la ruta o si no la creamos para guardar las imagenes cropped obtenidas de YOLOv5

In [91]:
if not os.path.exists(images_folder):
    os.makedirs(images_folder)

if not os.path.exists(labels_folder):
    os.makedirs(labels_folder)

Pipeline de YOLOv5 donde vamos a estar procesando las imagenes y obteniendo los objetos que sean detectados por el mismo modelo. Aqui tambien generamos la label para la imagen.

In [92]:
# detected_labels1 = []

# # Primera versión de detección de objetos con YOLO
# def detect_crop_and_label_v1(input_folder, images_folder, labels_folder):
#     images = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith('.jpg')]

#     for img_path in images:
#         img = cv2.imread(img_path)
#         results = model(img)

#         for i, (*xyxy, conf, cls) in enumerate(results.xyxy[0]):
#             x1, y1, x2, y2 = map(int, xyxy)
#             cropped_img = img[y1:y2, x1:x2]

#             label = int(cls)  # Obtener la clase predicha (label)
#             label_name = results.names[label]  # Obtener el nombre de la clase

#             # Guardar la imagen recortada en la subcarpeta de imágenes
#             output_filename = f"{os.path.basename(img_path).split('.')[0]}_{label_name}_{i}.jpg"
#             output_path = os.path.join(images_folder, output_filename)
#             cv2.imwrite(output_path, cropped_img)
#             print(f"Imagen recortada guardada en: {output_path}")

#             # Guardar la etiqueta en un archivo de texto en la subcarpeta de etiquetas
#             label_filename = f"{os.path.basename(img_path).split('.')[0]}_{label_name}_{i}.txt"
#             label_path = os.path.join(labels_folder, label_filename)
#             with open(label_path, 'w') as label_file:
#                 label_file.write(label_name)
#             print(f"Etiqueta guardada en: {label_path}")

#             detected_labels1.append(label_name)

In [93]:
# detect_crop_and_label_v1(input_folder, output_folder, labels_folder)

In [94]:
# print("Etiquetas detectadas durante la ejecución:")
# print(set(detected_labels1))  # Mostrar solo etiquetas únicas

In [95]:
# def detect_crop_and_label_v2(input_folder, images_folder, labels_folder):
#     images = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith('.jpg')]

#     for img_path in images:
#         img = cv2.imread(img_path)
#         results = model(img)

#         for i, (*xyxy, conf, cls) in enumerate(results.xyxy[0]):
#             label = int(cls)  # Obtener la clase predicha (label)
#             label_name = results.names[label]  # Obtener el nombre de la clase

#             # Guardar la etiqueta en un archivo de texto en la subcarpeta de etiquetas
#             label_filename = f"{os.path.basename(img_path).split('.')[0]}_{label_name}_{i}.txt"
#             label_path = os.path.join(labels_folder, label_filename)
#             with open(label_path, 'w') as label_file:
#                 label_file.write(label_name)
#             # print(f"Etiqueta guardada en: {label_path}")

#             detected_labels.append(label_name)

#             if label_name in classes_of_interest:
#                 x1, y1, x2, y2 = map(int, xyxy)
#                 cropped_img = img[y1:y2, x1:x2]

#                 # Guardar la imagen recortada en la subcarpeta de imágenes
#                 output_filename = f"{os.path.basename(img_path).split('.')[0]}_{label_name}_{i}.jpg"
#                 output_path = os.path.join(images_folder, output_filename)
#                 cv2.imwrite(output_path, cropped_img)
#                 # print(f"Imagen recortada guardada en: {output_path}")

#                 # Agregar la etiqueta detectada al array
#                 detected_labels.append(label_name)


Vamos a definir un tamaño estandarizado para las imagenes generadas.

In [96]:
# Definir el tamaño estandarizado de salida (por ejemplo, 256x256 píxeles)
standard_size = (256, 256)

In [97]:
detected_labels = []

def detect_crop_and_label_v3(input_folder, images_folder, labels_folder):
    images = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith('.jpg')]

    for img_path in images:
        img = cv2.imread(img_path)
        results = model(img)

        for i, (*xyxy, conf, cls) in enumerate(results.xyxy[0]):
            label = int(cls)  # Obtener la clase predicha (label)
            label_name = results.names[label]  # Obtener el nombre de la clase

            detected_labels.append(label_name)

            if label_name in classes_of_interest:
                # Reemplazar espacios en el nombre de la imagen
                img_name_cleaned = os.path.basename(img_path).split('.')[0].replace(' ', '_')

                # Guardar la etiqueta en un archivo de texto en la subcarpeta de etiquetas
                label_filename = f"{img_name_cleaned}_{label_name}_{i}.txt".replace(' ', '_')
                label_path = os.path.join(labels_folder, label_filename)
                with open(label_path, 'w') as label_file:
                    label_file.write(label_name)

                x1, y1, x2, y2 = map(int, xyxy)
                cropped_img = img[y1:y2, x1:x2]

                # Redimensionar la imagen recortada al tamaño estándar
                resized_img = cv2.resize(cropped_img, standard_size)

                # Guardar la imagen recortada y redimensionada en la subcarpeta de imágenes
                output_filename = f"{img_name_cleaned}_{label_name}_{i}.jpg".replace(' ', '_')
                output_path = os.path.join(images_folder, output_filename)
                cv2.imwrite(output_path, resized_img)
                # print(f"Imagen recortada y redimensionada guardada en: {output_path}")

    print(f"Proceso terminado para: {input_folder}")


Ejecutamos la función

In [98]:
input_folder_video_1 = 'datasets/dataset_secundario/Recorriendo CIUDAD DE GUATEMALA en vehículo (Parte 1) - GUATEMALA 2024'
input_folder_video_2 = 'datasets/dataset_secundario/Recorriendo CIUDAD DE GUATEMALA en vehículo (Parte 2) - GUATEMALA 2024'

In [99]:
detect_crop_and_label_v3(input_folder, images_folder, labels_folder)
detect_crop_and_label_v3(input_folder_video_1, images_folder, labels_folder)
detect_crop_and_label_v3(input_folder_video_2, images_folder, labels_folder)


Proceso terminado para: datasets/dataset_secundario
Proceso terminado para: datasets/dataset_secundario/Recorriendo CIUDAD DE GUATEMALA en vehículo (Parte 1) - GUATEMALA 2024
Proceso terminado para: datasets/dataset_secundario/Recorriendo CIUDAD DE GUATEMALA en vehículo (Parte 2) - GUATEMALA 2024


In [100]:
print("Etiquetas detectadas durante la ejecución:")
print(set(detected_labels))  # Mostrar solo etiquetas únicas

Etiquetas detectadas durante la ejecución:
{'potted plant', 'elephant', 'stop sign', 'vase', 'bus', 'cell phone', 'cow', 'laptop', 'sports ball', 'skateboard', 'boat', 'person', 'book', 'bench', 'bowl', 'banana', 'bird', 'bottle', 'backpack', 'clock', 'hot dog', 'bear', 'motorcycle', 'parking meter', 'tv', 'dog', 'giraffe', 'surfboard', 'traffic light', 'chair', 'suitcase', 'car', 'train', 'truck', 'umbrella', 'kite', 'keyboard', 'toilet', 'cup', 'fire hydrant', 'bicycle', 'cake', 'handbag', 'horse', 'airplane'}


<div class="alert alert-block alert-info">
<b>Posible mejora:</b> torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)
</div>

<div class="alert alert-block alert-info">
<b>Posible mejora:</b> Guardar imagenes en un formato de csv donde sea imagen, label, ruta. Para mejor eficiencia de manejo de espacio y tamaño de archivos.
</div>

<div class="alert alert-block alert-info">
<b>Posible mejora:</b> Fine tunning de YOLO.
</div>